In [122]:
import pandas as pd
import numpy as np


In [123]:
def reformat(data):
    '''
    wrangle stop Times data to get start and stop times between each stop
    '''
    
    reformattedData =  data.copy()
    tempTime= data['departure_time'].ix[1:].values
    times = pd.Series(tempTime)
    times.append(pd.Series(['2']))

    tempSequence = data['stop_sequence'].ix[1:].values
    sequence= pd.Series(tempSequence)
    sequence.append(pd.Series(['2']))

    start_id = data['stop_id'].ix[1:].values
    ids = pd.Series(start_id)
    ids.append(pd.Series(['2']))

    reformattedData['stop_sequence2'] =sequence
    reformattedData['departure_time'] = times
    reformattedData['stop_id'] = ids
    reformattedData['start_id'] = data['stop_id']
    
    goodIndexes = (reformattedData['stop_sequence'] +1 == reformattedData['stop_sequence2'])
    reformattedData = reformattedData[goodIndexes]
    return reformattedData
     
    

In [124]:
#helper functions for cleaning up time when day changes
def hh_mm_ss2seconds(hh_mm_ss):
    return reduce(lambda acc, x: acc*60 + x, map(int, hh_mm_ss.split(':')))

def cleanTimes(t):
    '''
    input: duration
    output: if duration < 0 , adds 24 hrs 
    '''
    if t<0:
        return t+ 24*60*60
    else :
        return t

In [125]:
#bus data for stops and stop times
brookStopTimes = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_brooklyn/stop_times.txt")
queensStopTimes = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_queens/stop_times.txt")
bronxStopTimes = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_bronx/stop_times.txt")
manhattanStopTimes = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_manhattan/stop_times.txt")
statStopTimes = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_staten_island/stop_times.txt")


brookStops = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_brooklyn/stops.txt")
queensStops = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_queens/stops.txt")
bronxStops = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_bronx/stops.txt")
manhattanStops = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_manhattan/stops.txt")
statStops = pd.read_csv("/home/michael/Desktop/bigDataProject/finalVersion/data/rawData/bus/google_transit_staten_island/stops.txt")



#all train data
stopTimes = pd.read_csv("trainData/stop_times.txt")
stops = pd.read_csv("trainData/stops.txt")
routes = pd.read_csv("trainData/routes.txt")
trips = pd.read_csv("trainData/trips.txt")
allRoutesTrain['type']="train"


#clean busTrips and trainTrips
bronx= reformat(bronxStopTimes)
stat= reformat(statStopTimes)
man =reformat(manhattanStopTimes)
queens = reformat(queensStopTimes)
brook = reformat(brookStopTimes)
allRoutesTrain = reformat(stopTimes)


allRoutesBus = pd.concat([bronx,stat,man,queens,brook])
allRoutesBus['type']="bus"

In [127]:

busStops= pd.concat([brookStops,queensStops,bronxStops,manhattanStops,statStops])
busStops['type']="bus"
stops['type']="train"



In [128]:
#combine all bus and train stops/trips
allStops = pd.concat([stops,busStops])
allRoutes = pd.concat([allRoutesTrain,allRoutesBus])



In [131]:

result = pd.merge(allRoutes, allStops, on='stop_id')
result.rename(columns = {'stop_lat':'dropoff_latitude' , 'stop_lon':'dropoff_longitude'}, inplace = True)
result2 = pd.merge(result, allStops, left_on='start_id',right_on='stop_id')
result2.rename(columns = {'stop_lat':'pickup_latitude' , 'stop_lon':'pickup_longitude'}, inplace = True)
result2= result2[['trip_id','stop_id_x','start_id','type','arrival_time','departure_time']]
result2.rename(columns = {'start_id':'mta_start_id','stop_id_x':'mta_stop_id'}, inplace = True)



In [134]:
#compute duration of trip
times = pd.DataFrame()
times['ar'] = result2['arrival_time'].apply(lambda x:hh_mm_ss2seconds(x) )
times['de'] = result2['departure_time'].apply(lambda x:hh_mm_ss2seconds(x) )
times['dif']= times['de']-times['ar']
#handle case where time stamp switched between days
result2['duration'] = times['dif'].apply(lambda x: cleanTimes(x)/60.)


,trip_id,mta_stop_id,mta_start_id,type,arrival_time,departure_time,duration
0,A20151206SUN_069000_GS.N03R,902N,901N,train,11:30:00,11:31:30,1.5
1,A20151206SUN_070000_GS.N03R,902N,901N,train,11:40:00,11:41:30,1.5
2,A20151206SUN_071000_GS.N03R,902N,901N,train,11:50:00,11:51:30,1.5
3,A20151206SUN_072000_GS.N03R,902N,901N,train,12:00:00,12:01:30,1.5
4,A20151206SUN_073000_GS.N03R,902N,901N,train,12:10:00,12:11:30,1.5


In [135]:
result2 = result2[['mta_stop_id','mta_stop_id','type','duration']]
result2.to_csv('/home/michael/Desktop/bigDataProject/finalVersion/data/mergedData/busTrainTrips.csv')

In [136]:
allStops.rename(columns = {'stop_id':'id','stop_lat':'lat','stop_lon':'long'}, inplace = True)
allStops =allStops[['id','lat','long','type']]


In [137]:
allStops.to_csv('/home/michael/Desktop/bigDataProject/finalVersion/data/mergedData/busTrainStops.csv')

In [140]:
def stopReducer(data,radius):
    '''
    performs one pass through the stop locations, combining stops that are within the radius 
    (manhattan distance). If a stop is assigned it is not eligible to be assigned to a new group.
    Basic function is to combine points that are close together
    '''
    reducerMap = dict()
    
    data['assigned'] = "a"
    groupNumber =0
    #data[mask]['assigned'].apply(lambda x:  False)
    rowNumber = 0
    while (data['assigned']=="a").sum()>0:  #continue until all of data has been assigned to a group
        row = data.iloc[rowNumber]
        if row['assigned']=="a":  # has not been assigned yet
            
            # create filter for "close points" so we don't have to look at all pairs
            data["x"] = data['lat'].apply(lambda x: np.absolute(x - row.lat))
            data["y"] = data['long'].apply(lambda x: np.absolute(x - row['long']))
            data['L1'] = data["x"] + data["y"]
            mask0= data['roughLatLon']==row.roughLatLon
            mask1 = data['L1']<radius
            
            #only want to combine with other points that have not been assigned
            mask2 = data['assigned']=="a"
            
            mask3 = np.logical_and(mask1, mask2)
            mask4 = np.logical_and(mask3,mask0)
            
            data.ix[mask4,'assigned']=groupNumber
            
            groupNumber += 1
        rowNumber +=1

    
    return   data


In [142]:
allStops['assigned']="a" # mark unassigned rows
allStops['roughLat']=allStops['lat'].apply(lambda x: float(int(x*100))/100)
allStops['roughLon']=allStops['long'].apply(lambda x: float(int(x*100))/100)
allStops['roughLatLon']=zip(allStops['roughLat'] , allStops['roughLon'])
allStops['assigned']="a"

a = stopReducer(allStops,.002)

In [144]:
reducedStops = a[['lat','long','type','assigned','id']]


In [153]:

def f(x):
    return pd.Series(dict(lat=x['lat'].mean(),
                            long=x['long'].mean(),
                            transtype="[%s]" % ', '.join(x['type'].unique())))



In [154]:
reducedStopsWithAveLatLon = reducedStops.groupby('assigned').apply(f)



In [159]:
reducedStops.head()

,lat,long,type,assigned,id
0,40.889248,-73.898583,train,0,101
1,40.889248,-73.898583,train,0,101N
2,40.889248,-73.898583,train,0,101S
3,40.884667,-73.900870,train,1,103
4,40.884667,-73.900870,train,1,103N


In [160]:
reducedStopsWithAveLatLon.to_csv('/home/michael/Desktop/bigDataProject/finalVersion/data/mergedData/reducedStops.csv')

In [157]:
reducedStops.shape

(14443, 5)